In [1]:
from collections import defaultdict 
import pandas as pd
import os
import math

In [2]:
"""PDF Generation Related Imports"""
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4,A3,A2,landscape
from reportlab.lib.units import inch,cm
from reportlab.lib import colors
import itertools

In [3]:
"""Functions Related to Dataframe savin in excel"""
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()

In [4]:
"""Functions Related to Color Genertation"""
def createColorList():
    mylist=[colors.aqua,colors.beige,colors.cyan,colors.khaki,colors.lawngreen,
            colors.lightcoral,colors.lightskyblue]
    return mylist

In [5]:
"""PDFgen related function for writing Resources """
def writeListInVerticalLineResources(txtlist,ylist,x,mycanvas,cellwidth,cellheight):
    xc=x+math.floor(cellwidth/2)
    print("wrting daily resources allocation data")
    print(txtlist)
    #print(vallist)
    for txt,y1 in zip(txtlist,ylist):
        if txt>0:            
            val1=txt*1000
            val2=math.ceil(val1/1000)
            if val2>10000:
                val3=val2/1000
                val4=val3*100
                val5=math.ceil(val4/100)
                mystr=str(val5)+"K"
            else:
                mystr=str(txt)
                mystr=str(val2)
        else:
            mystr="" 
        yc=y1+math.floor((1.5*cellheight)/2)
        mycanvas.drawCentredString(xc,yc,mystr)
def writeDailyAllocationDataResources(dframe,canvas_name,c_x,start_day,end_day,cellWidth,cellHeight):
    daytext=dayTextGenerator(start_day,end_day)
    x1=c_x
    c_y=1.0*inch
    delta=cellHeight
    fontsize=10
    """Setting fontsie and color"""
    canvas_name.setFont("Helvetica",fontsize)
    canvas_name.setFillColorRGB(0,0,0.77) 
    myclolor_list=createColorList()
    print(myclolor_list)
    for day in daytext:        
        vlist=list(dframe[day])
        vlist.insert(0,"")
        x2=x1+cellWidth
        xlist=[x1,x2]
        ylist=drawGridVertical(vlist,xlist,c_y,delta,canvas_name)
        fillGridVertical(vlist,xlist,c_y,delta,canvas_name,myclolor_list)
        canvas_name.setFillColorRGB(0,0,0)
        vlist1=vlist[1:]
        ylist1=ylist[1:]
        #writeListInVerticalLine(vlist1,ylist1,x1,canvas_name,cellWidth,cellHeight)
        writeListInVerticalLineResources(vlist1,ylist1,x1,canvas_name,cellWidth,cellHeight)
        x1+=cellWidth
        print(vlist)
def writeDayHeadingResources(start_day,finish_day,cellWidth,cellHeight,canvas_name,c_x,pno):
    daytext=dayTextGenerator(start_day,finish_day)
    print(daytext)
    if pno>1:
        start_day=start_day-(pno-1)*31
        finish_day=finish_day-(pno-1)*31
    multiplier=[i%32 for i in range(start_day,finish_day+1)]
    xlist=[c_x+(i-1)*cellWidth for i in multiplier]
    c_y=1.0*inch
    print("xlist={}".format(xlist))
    print("days={}".format(daytext))
    fontsize=8
    canvas_name.setFont("Helvetica",fontsize)
    #canvas_name.setFillColorRGB(1,1,1)
    writeListInHorizontalLine(daytext,c_y,xlist,canvas_name,cellWidth,cellHeight,1)
def writeHeadingResources(dframe,canvas_name):
    code=list(dframe['Code'])
    code.insert(0,"Code")
    names=list(dframe['Name'])
    names.insert(0,'Name')
    units=list(dframe['Unit'])
    units.insert(0,"Unit")
    #taskVolume=list(dframe['Task Vol'])
    #taskVolume.insert(0,"Task Volume")
    c_x=1.0*inch
    c_y=1.0*inch
    xlist=[1*inch,1.5*inch]
    delta=25
    fontsize=10
    """Setting fontsie and color"""
    canvas_name.setFont("Helvetica",fontsize)
    canvas_name.setFillColorRGB(0,0,0.77)    
    ylist=drawGridVertical(code,xlist,c_y,delta,canvas_name)
    #writeListVertical(code,c_x,c_y,canvas_name,25)
    writeListInVerticalLine(code,ylist,c_x,canvas_name,0.5*inch,25)
    c_x=1.5*inch
    xlist=[1.5*inch,3.5*inch]
    #writeListVertical(names,c_x,c_y,canvas_name,25)
    ylist=drawGridVertical(code,xlist,c_y,delta,canvas_name)
    writeListInVerticalLine(names,ylist,c_x,canvas_name,2.5*inch,25)
    c_x=3.5*inch
    xlist=[3.5*inch,4.7*inch]
    ylist=drawGridVertical(units,xlist,c_y,delta,canvas_name)
    writeListInVerticalLine(units,ylist,c_x,canvas_name,1.2*inch,25)
    #xlist=[4.7*inch,5.7*inch]
    #c_x=4.7*inch
    #ylist=drawGridVertical(taskVolume,xlist,c_y,delta,canvas_name)
    #writeListInVerticalLine(taskVolume,ylist,c_x,canvas_name,1*inch,25)    
    #c_x=3.0*inch
    #writeListVertical(units,c_x,c_y,canvas_name,25)
    return 4.7*inch
def writeProjectHeadingAtPageResources(canvas,project_name,start,finish):
    page_heading="Resources Allocation for "+project_name+"("+"D"+str(start)+"-"+"D"+str(finish)+")"    
    yc=0.95*inch
    xc=8.25*inch
    fontsize=14
    canvas.setFont("Helvetica",fontsize)
    canvas.setFillColorRGB(0.87,0.37,0.77)
    canvas.drawCentredString(xc,yc,page_heading)
 #writeProjectHeadingAtPageResources
def writeResourceAllocationReport(project_name,task_df,filename,work_graph):
    print("sucessfully initiated report writing....... ")
    """Setting UP """
    myvalue=createPage(filename,A3)
    page=myvalue["canvas"]
    c_x=myvalue["c_x"]
    c_y=myvalue["c_y"]
    """Dividing Pages"""
    #start=min(list(myframe['Start']))
    start=1
    finish=work_graph.finishDay
    #finish=max(list(myframe['Finish']))
    no_of_pages=math.ceil(finish/31)
    page_start=[1+i*31 for i in range(0,no_of_pages)]
    page_finish=[x-1+31 for x in page_start]
    #page_finish[-1]=finish
    page_no=1
    project_name=work_graph.siteName
    print("start={} finish={}".format(page_start,page_finish))
    for sindex,findex in zip(page_start,page_finish):
        writeProjectHeadingAtPageResources(page,project_name,sindex,findex)
        c_x=writeHeadingResources(task_df,page)
        #c_x=writeHeading(task_df,page)
        writeDayHeadingResources(sindex,findex,25,25,page,c_x,page_no)
        writeDailyAllocationDataResources(task_df,page,c_x,sindex,findex,25,25)
        page.showPage()
        page_no+=1
    """Writing test list"""
    #mylist=list(allocation_df['Code'])
    #mylist.insert(0,"")
    #writeListVertical(mylist,c_x,c_y,page,25)
    #c_x=writeHeading2(task_df,page)
    #writeDayHeading(1,20,25,25,page,c_x)writeDayHeadingResources
    #page.drawString("Ok",100,100)
    #writeDailyAllocationData(allocation_df,page,c_x,1,20,25,25)

    #page.showPage()
    page.save()
def writeResourceAllocationReportSummary(project_name,task_df,filename,finish_day):
    print("sucessfully initiated report writing....... ")
    """Setting UP """
    myvalue=createPage(filename,A3)
    page=myvalue["canvas"]
    c_x=myvalue["c_x"]
    c_y=myvalue["c_y"]
    """Dividing Pages"""
    #start=min(list(myframe['Start']))
    start=1
    finish=finish_day-3
    #finish=max(list(myframe['Finish']))
    no_of_pages=math.ceil(finish/31)
    page_start=[1+i*31 for i in range(0,no_of_pages)]
    page_finish=[x-1+31 for x in page_start]
    #page_finish[-1]=finish
    page_no=1
    project_name=project_name
    print("start={} finish={}".format(page_start,page_finish))
    for sindex,findex in zip(page_start,page_finish):
        writeProjectHeadingAtPageResources(page,project_name,sindex,findex)
        c_x=writeHeadingResources(task_df,page)
        #c_x=writeHeading(task_df,page)
        writeDayHeadingResources(sindex,findex,25,25,page,c_x,page_no)
        writeDailyAllocationDataResources(task_df,page,c_x,sindex,findex,25,25)
        page.showPage()
        page_no+=1
    """Writing test list"""
    #mylist=list(allocation_df['Code'])
    #mylist.insert(0,"")
    #writeListVertical(mylist,c_x,c_y,page,25)
    #c_x=writeHeading2(task_df,page)
    #writeDayHeading(1,20,25,25,page,c_x)writeDayHeadingResources
    #page.drawString("Ok",100,100)
    #writeDailyAllocationData(allocation_df,page,c_x,1,20,25,25)

    #page.showPage()
    page.save()

In [6]:
"""PDFgen related function for writing tasks """
def createPage_phased_out(out_file_name):    
    mycanvas=canvas.Canvas(out_file_name,pagesize=landscape(A4),bottomup =0)
    width, height = landscape(A4)
    width=width-1*inch
    height=height-1*inch
    c_x=1*inch
    c_y=1*inch
    myvalue={"width":width,"height":height,"canvas":mycanvas,'c_x':c_x,'c_y':c_y}
    return myvalue
def createPage(out_file_name,page_size=A4):    
    mycanvas=canvas.Canvas(out_file_name,pagesize=landscape(page_size),bottomup =0)
    width, height = landscape(page_size)
    width=width-1*inch
    height=height-1*inch
    c_x=1*inch
    c_y=1*inch
    myvalue={"width":width,"height":height,"canvas":mycanvas,'c_x':c_x,'c_y':c_y}
    return myvalue
def writeListVertical(mylist,c_x,c_y,mycanvas,dely):
    #dely=25
    #delx=25
    y=c_y
    x=c_x    
    mycanvas.setFont("Helvetica",8)
    mycanvas.setFillColorRGB(0,0,0.77)
    for txt in mylist:        
        mycanvas.drawString(x,y,str(txt))
        y +=dely
def writeListInVerticalLine(txtlist,ylist,x,mycanvas,cellwidth,cellheight):
    xc=x+math.floor(cellwidth/2)
    for txt,y1 in zip(txtlist,ylist):
        yc=y1+math.floor((1.5*cellheight)/2)
        mycanvas.drawCentredString(xc,yc,str(txt))
def writeListInVerticalLineAllocation(txtlist,ylist,x,mycanvas,cellwidth,cellheight):
    xc=x+math.floor(cellwidth/2)    
    #print(txtlist)
    #print(vallist)
    for txt,y1 in zip(txtlist,ylist):
        if txt>0:
            mystr=str(txt)
        else:
            mystr="" 
        yc=y1+math.floor((1.5*cellheight)/2)
        mycanvas.drawCentredString(xc,yc,mystr)
        
def writeListInHorizontalLine(txtlist,y,xlist,mycanvas,cellwidth,cellheight,text_align):
    yc=y+math.floor(cellheight/2)
    for txt,x1 in zip(txtlist,xlist):
        xc=x1+math.floor((1.0*cellwidth)/2) 
        mystr=str(txt)
        if text_align==1:            
            mycanvas.drawCentredString(xc,yc,mystr)
        else:
            mycanvas.drawString(xc,yc,mystr)
        
def writeListHorizontal(mylist,c_x,c_y,mycanvas,delx):
    #dely=25
    #delx=25
    y=c_y
    x=c_x
    mycanvas.setFont("Helvetica",fontsize)
    mycanvas.setFillColorRGB(0,0,0.77)    
    for txt in mylist:        
        mycanvas.drawString(x,y,str(txt))
        x +=delx
def drawGridVertical(mylist,xlist,c_y,delta,mycanvas):
    ylist=[c_y+i*delta for i in range(len(mylist)+1)]
    mycanvas.grid(xlist,ylist)
    return ylist

def fillGridVertical(mylist,xlist,c_y,delta,mycanvas,colorlist):
    ylist1=[c_y+i*delta for i in range(len(mylist)+1)]
    ylist=ylist1[1:]
    color_index=[i%7 for i in range(0,len(ylist))]
    mycolors=[colorlist[i] for i in color_index]
    x=xlist[0] 
    vallist=mylist[1:]
    for val,y,fill_color in zip(vallist,ylist,mycolors):
        if val >0:
            mycanvas.setFillColor(fill_color)
            mycanvas.rect(x,y,delta,delta,fill=1)
        
#Heading for Task Allocation    
def writeHeading(dframe,canvas_name):
    code=list(dframe['TaskNo'])
    code.insert(0,"TaskNo")
    names=list(dframe['Task_Name'])
    names.insert(0,'Task_Name')
    units=list(dframe['Unit'])
    units.insert(0,"Unit")
    taskVolume=list(dframe['Task Vol'])
    taskVolume.insert(0,"Task Volume")
    c_x=1.0*inch
    c_y=1.0*inch
    xlist=[1*inch,1.5*inch]
    delta=25
    fontsize=9
    """Setting fontsie and color"""
    canvas_name.setFont("Helvetica",fontsize)
    canvas_name.setFillColorRGB(0,0,0.77)    
    ylist=drawGridVertical(code,xlist,c_y,delta,canvas_name)
    #writeListVertical(code,c_x,c_y,canvas_name,25)
    writeListInVerticalLine(code,ylist,c_x,canvas_name,0.5*inch,25)
    c_x=1.5*inch
    xlist=[1.5*inch,4.0*inch]
    #writeListVertical(names,c_x,c_y,canvas_name,25)
    ylist=drawGridVertical(code,xlist,c_y,delta,canvas_name)
    writeListInVerticalLine(names,ylist,c_x,canvas_name,2.5*inch,25)
    c_x=4.0*inch
    xlist=[4.0*inch,4.7*inch]
    ylist=drawGridVertical(units,xlist,c_y,delta,canvas_name)
    writeListInVerticalLine(units,ylist,c_x,canvas_name,0.7*inch,25)
    xlist=[4.7*inch,5.7*inch]
    c_x=4.7*inch
    ylist=drawGridVertical(taskVolume,xlist,c_y,delta,canvas_name)
    writeListInVerticalLine(taskVolume,ylist,c_x,canvas_name,1*inch,25)
    
    
    #c_x=3.0*inch
    #writeListVertical(units,c_x,c_y,canvas_name,25)
    return 5.7*inch
#Heading for Resource Allocation








def writeDayHeading(start_day,finish_day,cellWidth,cellHeight,canvas_name,c_x,pno):
    daytext=dayTextGenerator(start_day,finish_day)
    print(daytext)
    if pno>1:
        start_day=start_day-(pno-1)*31
        finish_day=finish_day-(pno-1)*31
    multiplier=[i%32 for i in range(start_day,finish_day+1)]
    xlist=[c_x+(i-1)*cellWidth for i in multiplier]
    c_y=1.0*inch
    print("xlist={}".format(xlist))
    print("days={}".format(daytext))
    fontsize=8
    canvas_name.setFont("Helvetica",fontsize)
    #canvas_name.setFillColorRGB(1,1,1)
    writeListInHorizontalLine(daytext,c_y,xlist,canvas_name,cellWidth,cellHeight,1)
    
def writeDailyAllocationData(dframe,canvas_name,c_x,start_day,end_day,cellWidth,cellHeight):
    daytext=dayTextGenerator(start_day,end_day)
    x1=c_x
    c_y=1.0*inch
    delta=cellHeight
    fontsize=10
    """Setting fontsie and color"""
    canvas_name.setFont("Helvetica",fontsize)
    canvas_name.setFillColorRGB(0,0,0.77) 
    myclolor_list=createColorList()
    print(myclolor_list)
    for day in daytext:        
        vlist=list(dframe[day])
        vlist.insert(0,"")
        x2=x1+cellWidth
        xlist=[x1,x2]
        ylist=drawGridVertical(vlist,xlist,c_y,delta,canvas_name)
        fillGridVertical(vlist,xlist,c_y,delta,canvas_name,myclolor_list)
        canvas_name.setFillColorRGB(0,0,0)
        vlist1=vlist[1:]
        ylist1=ylist[1:]
        writeListInVerticalLineAllocation(vlist1,ylist1,x1,canvas_name,cellWidth,cellHeight)
        x1+=cellWidth
        print(vlist)

def writeProjectHeadingAtPage(canvas,project_name,start,finish):
    page_heading="Work Program for "+project_name+"("+"D"+str(start)+"-"+"D"+str(finish)+")"    
    yc=0.95*inch
    xc=8.25*inch
    fontsize=14
    canvas.setFont("Helvetica",fontsize)
    canvas.setFillColorRGB(0.87,0.37,0.77)
    canvas.drawCentredString(xc,yc,page_heading)

    
    #pass
def writeTaskAllocationReport(project_name,task_df,filename,work_graph):
    print("sucessfully initiated report writing....... ")
    """Setting UP """
    myvalue=createPage(filename,A3)
    page=myvalue["canvas"]
    c_x=myvalue["c_x"]
    c_y=myvalue["c_y"]
    """Dividing Pages"""
    #start=min(list(myframe['Start']))
    start=1
    finish=work_graph.finishDay
    #finish=max(list(myframe['Finish']))
    no_of_pages=math.ceil(finish/31)
    page_start=[1+i*31 for i in range(0,no_of_pages)]
    page_finish=[x-1+31 for x in page_start]
    #page_finish[-1]=finish
    page_no=1
    project_name=work_graph.siteName
    print("start={} finish={}".format(page_start,page_finish))
    for sindex,findex in zip(page_start,page_finish):
        writeProjectHeadingAtPage(page,project_name,sindex,findex)
        c_x=writeHeading(task_df,page)
        writeDayHeading(sindex,findex,25,25,page,c_x,page_no)
        writeDailyAllocationData(task_df,page,c_x,sindex,findex,25,25)
        page.showPage()
        page_no+=1
    """Writing test list"""
    #mylist=list(allocation_df['Code'])
    #mylist.insert(0,"")
    #writeListVertical(mylist,c_x,c_y,page,25)
    #c_x=writeHeading2(task_df,page)
    #writeDayHeading(1,20,25,25,page,c_x)
    #page.drawString("Ok",100,100)
    #writeDailyAllocationData(allocation_df,page,c_x,1,20,25,25)

    #page.showPage()
    page.save()
    
"""Writing Combined Resources Frame"""  
    
def writeTaskAllocationReportCombined(project_name,task_df,filename,finish_day):
    print("sucessfully initiated report writing....... ")
    """Setting UP """
    myvalue=createPage(filename,A3)
    page=myvalue["canvas"]
    c_x=myvalue["c_x"]
    c_y=myvalue["c_y"]
    """Dividing Pages"""
    #start=min(list(myframe['Start']))
    start=1
    #finish=work_graph.finishDay
    finish=finish_day
    #finish=max(list(myframe['Finish']))
    no_of_pages=math.ceil(finish/31)
    page_start=[1+i*31 for i in range(0,no_of_pages)]
    page_finish=[x-1+31 for x in page_start]
    #page_finish[-1]=finish
    page_no=1
    project_name=project_name
    print("start={} finish={}".format(page_start,page_finish))
    for sindex,findex in zip(page_start,page_finish):
        writeProjectHeadingAtPage(page,project_name,sindex,findex)
        c_x=writeHeading(task_df,page)
        writeDayHeading(sindex,findex,25,25,page,c_x,page_no)
        writeDailyAllocationData(task_df,page,c_x,sindex,findex,25,25)
        page.showPage()
        page_no+=1
    """Writing test list"""
    #mylist=list(allocation_df['Code'])
    #mylist.insert(0,"")
    #writeListVertical(mylist,c_x,c_y,page,25)
    #c_x=writeHeading2(task_df,page)
    #writeDayHeading(1,20,25,25,page,c_x)
    #page.drawString("Ok",100,100)
    #writeDailyAllocationData(allocation_df,page,c_x,1,20,25,25)

    #page.showPage()
    page.save()


    
    

In [7]:
class GanttBar:
    def __init__(self,start,duration,name="",unit="",volume=0):
        self.taskName=name
        self.start_day=start
        self.duration=duration
        self.finish_day=self.start_day+self.duration-1
        self.taskUnit=unit
        self.taskVolume=volume
        self.productionRate=math.ceil(self.taskVolume/self.duration)
    def addTaskUnit(self,unit):
        self.taskUnit=unit
    def addTaskVolume(self,volume):
        self.taskVolume=volume
        self.productionRate=math.ceil(self.taskVolume/self.duration)
    def addResources(self,consum_df):
        cols=consum_df.columns
        unit_resources=list(consum_df[self.taskCodeNo])
        consumption=[x*self.productionRate for x in unit_resources] 
        consumption[-1]= (math.ceil(consumption[-1]/1000.00))
        self.consumptionRate= consumption

        print("unit resources={} total_consumption={}".format( unit_resources,consumption))
    def addTaskCode(self,codeNo):
        self.taskCodeNo=codeNo
        

In [8]:
"""Graph Driver"""
def text2List(input_text):
    split_text=input_text.split(',')
    output_list=[int(x) for x in split_text ]
    return output_list
    
def buildGraph(input_df):
    shape=input_df.shape
    g=Graph(shape[0])
    print(shape)
    for index,row in input_df.iterrows():
        v=row['TaskNo']
        if row['Predecessor']!=-1:
            pred_text=str(row['Predecessor'])
            length=len(pred_text)
            if length >1:
                sources=text2List(pred_text)
                #print(sources)
            else:
                sources=[int(pred_text)]
            #print("sources={}".format(sources))
            for u in sources:
                g.addEdge(u,v)
            #print("predecosseo no={} value={}".format(length,pred_text))
            #print(len(pred_text))
            #print(pred_text)
    duration_list=list(input_df['Duration'])
    g.addDurations(duration_list)
    return g


In [9]:
"""Class to represent graph"""
class Graph:
    def __init__(self,vertices):
        self.graph=defaultdict(list)
        self.V=vertices
        self.indeg0=[True]*(self.V+1)
        self.outdeg0=[True]*(self.V+1)
        self.edges=[]
        self.adj=defaultdict(list)
        self.visited=[False]*(self.V+1)
        self.path=[]
        self.durations=[]
        self.taskchains=[]
        self.tc_durations=[]
        self.minimum_start_date=[0]*self.V
    # function to add an edge to graph 
    def addEdge(self,u,v):
        self.graph[u].append(v)
        edge=(u,v)
        print(edge)
        self.edges.append(edge)
        # set indeg0[v] <- false 
        self.indeg0[v] = False  
        # set outdeg0[u] <- false 
        self.outdeg0[u] = False
        #print(self.graph[u])
    # A recursive function used by topologicalSort 
    def topologicalSortUtil(self,v,visited,stack):
        #mark the current node as visited
        visited[v]=True
        # Recur for all the vertices adjacent to this vertex      

        for i in self.graph[v]:
            if visited[i]==False:
                self.topologicalSortUtil(i,visited,stack)
         # Mark the current node as visited. 
        stack.insert(0,v)
    # The function to do Topological Sort. It uses recursive  
    # topologicalSortUtil() 
    def topologicalSort(self):
        # Mark all the vertices as not visited
        visited=[False]*(self.V+1)
        stack=[]
        # Call the recursive helper function to store Topological 
        # Sort starting from all vertices one by one 
        for i in range(self.V):
            if visited[i]==False:
                self.topologicalSortUtil(i,visited,stack)
        print (stack)
    def displayGraph(self):
        print("number of vertices={}".format(self.V))
        print(self.graph)
        print("indegree={}".format(self.indeg0))
        print("outdegree={}".format(self.outdeg0))
        print("edges of the graph......")
        print(self.edges)
        print("durations of tasks")
        print(self.durations)
    def dfs(self,s):
        k=0       
        #append the node in path
        #and set visited
        self.path.append(s)
        self.visited[s]=True
        # Path started with a node 
        # having in-degree 0 and 
        # current node has out-degree 0, 
        # print current path 
        if self.outdeg0[s] and self.indeg0[self.path[0]]:
            print(*self.path)
            self.taskchains.append(list(self.path))
            #for p in self.path:
                #self.taskchains[k].append(p)               
            #myvalue[k]=self.path
            #
        k=k+1
            #print(myvalue)
        # Recursive call to print all paths
        for node in self.graph[s]:
            if not self.visited[node]:
                self.dfs(node)
        # Remove node from path 
        # and set unvisited
        #return self.path
        #print(self.path)
        #self.taskchains[s]=self.path
        self.path.pop()
        self.visited[s]=False
        #return myvalue
    def print_all_paths(self):
        for i in range(self.V):
            if self.indeg0[i] and self.graph[i]:
                self.path=[]
                self.visited=[False]*(self.V+1)
                self.dfs(i)
                #print("path={}".format(p))
        
    def addDurations(self,duration_list):
        self.durations=duration_list
        
    def findPathdurations(self,path):
        total_duration=0
        for p in path:
            total_duration +=self.durations[p-1]
        return total_duration
    def findPathCumulativeDurations(self,path):
        total_duration=0
        path_cum=[]
        for p in path:
            total_duration +=self.durations[p-1]
            path_cum.append(total_duration) 
        self.tc_durations.append(path_cum)
        #return path_cum
    def findStratDateofTasks(self):
        for T in range(1,self.V+1):
            d=self.findMinimumStartDateOfTask(T)
            print("Task-{} sdate={}".format(T,d))
            self.minimum_start_date[T-1]=d
    def findMinimumStartDateOfTask(self,T):
        d=self.durations[T-1]
        duration_list=[]
        for tc,tcd in zip(self.taskchains,self.tc_durations):
            if T in tc:
                index=tc.index(T)
                sdate=tcd[index]-d+1
                duration_list.append(sdate)
                #print("task found in={}".format(index))
            #print("chain={} duration={}".format(tc,tcd))
        #print("duration of tasks no={} total_time={}".format(T,d))
        max_start_date=max(duration_list) # one is added to prevent overlaping of bars       
        return max_start_date
    def calculateStartOfAllTasks(self):
        for chain in self.taskchains:
            #duration=self.findPathdurations(chain)
            self.findPathCumulativeDurations(chain)
           
    def scheduleTask(self):
        self.topologicalSort() 
        #self.displayGraph()
        self.print_all_paths()
        print(self.taskchains)
        for chain in self.taskchains:
            #duration=self.findPathdurations(chain)
            self.findPathCumulativeDurations(chain)
        #self.calculateStartOfAllTasks()
        #print(self.tc_durations) 
        self.findStratDateofTasks() 
        self.findProjectDuration()
    def createGanttSchedule(self,names,units,volumes,tocdes,df_unit_res):
        mytasks=[]
        for i in range(1,self.V+1):            
            duration=self.durations[i-1]
            sday=self.minimum_start_date[i-1]
            unit=units[i-1]
            volume=volumes[i-1]
            name=names[i-1]
            bar=GanttBar(sday,duration,name,unit,volume)
            tcode=tcodes[i-1]
            bar.addTaskCode(tcode)
            bar.addResources(df_unit_res)
            
            mytasks.append(bar)
        return mytasks
    def findProjectDuration(self):
        project_durations=[]
        for tcd in self.tc_durations:
            dmax=max(tcd)
            #print(dmax)
            project_durations.append(dmax+1)
        self.finishDay=max(project_durations)+1
    def setProjectName(self,project_name):
        self.siteName=project_name
        
            
            
            
        
   
   
        
        


In [10]:
"""Task Frame Related Function"""
def dayTextGenerator(start,finish):
    gridtext=["D"+str(i)  for i in range(start,finish+1)]
    return gridtext
def populateWorkMatrix(mybars,alloc_df):
    work_name_list=list(alloc_df['Task_Name'])
    column_name_list=list(alloc_df.columns)
    for bar in mybars:
        start_day=bar.start_day
        finish_day=bar.finish_day
        daytext=dayTextGenerator(start_day,finish_day)
        row=work_name_list.index(bar.taskName)
        prt=bar.productionRate
        for txt in daytext:
            col=column_name_list.index(txt)
            alloc_df.iloc[row,col]=prt
    return alloc_df    
def populateResourceMatrix(mybars,resource_df):
    #work_name_list=list(alloc_df['Task_Name'])
    column_name_list=list(resource_df.columns)
    for bar in mybars:
        start_day=bar.start_day
        finish_day=bar.finish_day
        daytext=dayTextGenerator(start_day,finish_day)
        #row=work_name_list.index(bar.taskName)
        #prt=bar.productionRate
        for txt in daytext:
            #col=column_name_list.index(txt)
            cum_res=list(resource_df[txt])
            cur_res=bar.consumptionRate
            added_res=[cum_res[i]+cur_res[i]for i in range(len(cum_res))]
            resource_df[txt]=added_res
            
    return resource_df  
def roundResourcesAllocationMatrix(resource_df):
    myshape=resource_df.shape
    for i in range( myshape[0]-1):
        for j in range(3,myshape[1]):
            resource_df.iloc[i,j]=math.ceil(resource_df.iloc[i,j]/1.0)
    return resource_df
            
    
    
def buildWorkMatrix(mybars,graph):
    work_no=[i+1 for i in range(len(mybars))]
    zero_list=[0 for i in range(len(mybars))]
    work_name=[]
    work_duration=[]
    work_unit=[]
    work_volume=[]
    work_prt=[]
    work_start=[]
    work_finish=[]  
    for bar in mybars:
        work_name.append(bar.taskName)
        work_unit.append(bar.taskUnit)
        work_volume.append(bar.taskVolume)
        work_prt.append(bar.productionRate)
        work_duration.append(bar.duration)
        work_start.append(bar.start_day)
        work_finish.append(bar.finish_day)
        
    data={'TaskNo':work_no,'Task_Name':work_name,'Start Day':work_start,'Finish Day':work_finish,'Durations':work_duration ,'Unit':work_unit,'Task Vol': work_volume,'Work_rate':work_prt}
    task_allocation_df=pd.DataFrame(data)
    finish_day_multiplier=math.ceil(graph.finishDay/31) # homany 31 days
    finish_day=finish_day_multiplier*31    

    daytext=dayTextGenerator(1,finish_day)
    k=6
    for text in daytext:
        task_allocation_df.insert(k,text,zero_list,True)
        k=k+1
    task_allocation_df=populateWorkMatrix(mybars,task_allocation_df)
    return task_allocation_df
def buildResoucesAllocationMatrix(mybars,resources_df,graph):
    codelist=list(resources_df['Code'])
    rname=list(resources_df['Name'])
    units=list(resources_df['Unit'])  
    data={'Code':codelist,'Name':rname,'Unit':units}
    resources_allocation_df=pd.DataFrame(data)
    finish_day_multiplier=math.ceil(graph.finishDay/31) # homany 31 days
    finish_day=finish_day_multiplier*31     
    daytext=dayTextGenerator(1,finish_day)
    zero_list=[0 for i in range(len(codelist))]    
    k=3
    for text in daytext:
        resources_allocation_df.insert(k,text,zero_list,True)
        k=k+1
    
    resources_allocation_df=populateResourceMatrix(mybars,resources_allocation_df)
    resources_allocation_df=roundResourcesAllocationMatrix(resources_allocation_df)
    return resources_allocation_df
def findMaximumDurations(rframes):
    max_val=0
    for df in rframes:
        if max_val< df.shape[1]:
            max_val=df.shape[1]  
        print("shape={}".format(df.shape))
    print("max={}".format(max_val))
    return max_val
def addDatatoCombinedFrame(current_frame,total_frame,fday):
    daytext=dayTextGenerator(1,fday)
    print("days={}".format(daytext))
    for txt in daytext:
            #col=column_name_list.index(txt)
            cum_res=list(total_frame[txt])
            cur_res=list(current_frame[txt])
            added_res=[cum_res[i]+cur_res[i]for i in range(len(cum_res))]
            #added_res=cur_res
            total_frame[txt]=added_res    
    return total_frame
    
def buildCombinedResourceMatrix(rframes):
    max_val=findMaximumDurations(resoureces_frames)    
    tframe=rframes[0]
    cols=tframe.columns
    codelist=list(tframe[cols[0]])
    namelist=list(tframe[cols[1]])
    unitlist=list(tframe[cols[2]])
    data={"Code": codelist,"Name": namelist,"Unit":unitlist}
    combined_resource_df=pd.DataFrame(data)
    zero_list=[0 for i in range(len(codelist))] 
    daytext=dayTextGenerator(1,max_val)
    k=3
    for text in daytext:
        combined_resource_df.insert(k,text,zero_list,True)
        k=k+1
    for cframe in rframes:
        fday=cframe.shape[1]-3
        combined_resource_df=addDatatoCombinedFrame(cframe,combined_resource_df,fday)
    return combined_resource_df
   
    
    
    
        
        
        
    
  
        

In [11]:
#myfolder=r'F:\website\cmis6\Civilworks cost\Work Schedule' #office computer
myfolder=r'E:\Website_26_07_2020\cmis6\Civilworks cost\Work Schedule' #home computer
inputpath=os.path.join(myfolder,'Schdule Input.xlsx')
outputpath_for_dframe=os.path.join(myfolder,'dframes.xlsx')
outpath_for_resources=os.path.join(myfolder,'resources_frame.xlsx')
sheetName="Resources_Consumption"
unit_resource_df=pd.read_excel(inputpath,sheet_name=sheetName)
resources_sheet_names=[]
myframes=[]
mynames=[]
resoureces_frames=[]
sheetName="Main"
main_df=pd.read_excel(inputpath,sheet_name=sheetName)
main_df
for index,row in main_df.iterrows():
    sheetName=row['Allocation_Sheet_Name']
    task_df=pd.read_excel(inputpath,sheet_name=sheetName)
    task_df.fillna(0,inplace=True)
    g=buildGraph(task_df)
    g. scheduleTask()
    project_name=row['Site Name']
    g.setProjectName(project_name)
    print(g.finishDay)
    tnames=list(task_df['Name'])
    units=list(task_df['Unit'])
    volumes=list(task_df['Volume'])
    tcodes=list(task_df['TaskCode'])
    print("unit={} volumes={} codes={}".format(tnames,units,volumes,tcodes))
    mybars=g.createGanttSchedule(tnames,units,volumes,tcodes,unit_resource_df)
    g.findProjectDuration()
    #print(task_df)
    print("before adding unit......")
      
    for bar in mybars:
        print("name={} unit={} volume={} production={}".format(bar.taskName,bar.taskUnit,
                                                              bar.taskVolume,bar.productionRate))
    """Building Works Allocation Matrix"""
    work_alloc_df=buildWorkMatrix(mybars,g)
    resources_alloc_df= buildResoucesAllocationMatrix(mybars,unit_resource_df,g)
    "Wrting work schedule"
    file_name=sheetName+"_W.pdf"
    full_file_name=os.path.join(myfolder, file_name)
    writeTaskAllocationReport(sheetName,work_alloc_df,full_file_name,g) 
    "Wrting resources schedule"
    file_name2=sheetName+"_R.pdf"
    full_file_name=os.path.join(myfolder, file_name2)
    writeResourceAllocationReport(sheetName,resources_alloc_df,full_file_name,g)  
    
    myframes.append(task_df)
    mynames.append(sheetName)
    myframes.append(work_alloc_df)
    mynames.append(sheetName+"-mat")
    """Adding Resources Allocation Df"""
    resoureces_frames.append(resources_alloc_df)
    resources_sheet_names.append(sheetName+"_R")    
    resoureces_frames
     
    




(10, 7)
(1, 2)
(1, 3)
(2, 4)
(3, 4)
(1, 5)
(4, 6)
(5, 6)
(4, 7)
(5, 7)
(4, 8)
(5, 8)
(4, 9)
(5, 9)
(9, 10)
[1, 5, 3, 2, 4, 9, 10, 8, 7, 6, 0]
1 2 4 6
1 2 4 7
1 2 4 8
1 2 4 9 10
1 3 4 6
1 3 4 7
1 3 4 8
1 3 4 9 10
1 5 6
1 5 7
1 5 8
1 5 9 10
[[1, 2, 4, 6], [1, 2, 4, 7], [1, 2, 4, 8], [1, 2, 4, 9, 10], [1, 3, 4, 6], [1, 3, 4, 7], [1, 3, 4, 8], [1, 3, 4, 9, 10], [1, 5, 6], [1, 5, 7], [1, 5, 8], [1, 5, 9, 10]]
Task-1 sdate=1
Task-2 sdate=8
Task-3 sdate=8
Task-4 sdate=29
Task-5 sdate=8
Task-6 sdate=50
Task-7 sdate=50
Task-8 sdate=50
Task-9 sdate=50
Task-10 sdate=67
75
unit=['Mobilization and Site Preparation', 'CC Block Manufacture(30X30X30)', 'CC Block Manufacture(40X40X20)', 'Curing CC Block', 'Embankment Construction', 'Placing Sand Filter', 'Placing Geotextile Filter', 'Placing Pea Gravel Filter', 'Placing CC Block', 'Ancilary Work for Site Completion'] volumes=['Percent', 'Nos', 'Nos', 'Percent', 'Cum', 'Cum', 'sqm', 'Cum', 'Nos', 'Percent'] codes=[100, 21058, 55449, 100, 7388, 918, 1076

(10, 7)
(1, 2)
(1, 3)
(2, 4)
(3, 4)
(1, 5)
(4, 6)
(5, 6)
(4, 7)
(5, 7)
(4, 8)
(5, 8)
(4, 9)
(5, 9)
(9, 10)
[1, 5, 3, 2, 4, 9, 10, 8, 7, 6, 0]
1 2 4 6
1 2 4 7
1 2 4 8
1 2 4 9 10
1 3 4 6
1 3 4 7
1 3 4 8
1 3 4 9 10
1 5 6
1 5 7
1 5 8
1 5 9 10
[[1, 2, 4, 6], [1, 2, 4, 7], [1, 2, 4, 8], [1, 2, 4, 9, 10], [1, 3, 4, 6], [1, 3, 4, 7], [1, 3, 4, 8], [1, 3, 4, 9, 10], [1, 5, 6], [1, 5, 7], [1, 5, 8], [1, 5, 9, 10]]
Task-1 sdate=1
Task-2 sdate=8
Task-3 sdate=8
Task-4 sdate=29
Task-5 sdate=8
Task-6 sdate=50
Task-7 sdate=50
Task-8 sdate=50
Task-9 sdate=50
Task-10 sdate=67
75
unit=['Mobilization and Site Preparation', 'CC Block Manufacture(30X30X30)', 'CC Block Manufacture(40X40X20)', 'Curing CC Block', 'Embankment Construction', 'Placing Sand Filter', 'Placing Geotextile Filter', 'Placing Pea Gravel Filter', 'Placing CC Block', 'Ancilary Work for Site Completion'] volumes=['Percent', 'Nos', 'Nos', 'Percent', 'Cum', 'Cum', 'sqm', 'Cum', 'Nos', 'Percent'] codes=[100, 21058, 55449, 100, 7388, 918, 1076

wrting daily resources allocation data
[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 13.0]
['', 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 13.0]
wrting daily resources allocation data
[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 13.0]
['', 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 13.0]
wrting daily resources allocation data
[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 13.0]
['', 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 13.0]
wrting daily resources allocation data
[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 13.0]
['', 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 13.0]
wrting daily resources allocation data
[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 13.0]
['', 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 21.0, 13.0]
wrtin

Task-2 sdate=8
Task-3 sdate=8
Task-4 sdate=29
Task-5 sdate=8
Task-6 sdate=50
Task-7 sdate=50
Task-8 sdate=50
Task-9 sdate=50
Task-10 sdate=67
75
unit=['Mobilization and Site Preparation', 'CC Block Manufacture(30X30X30)', 'CC Block Manufacture(40X40X20)', 'Curing CC Block', 'Embankment Construction', 'Placing Sand Filter', 'Placing Geotextile Filter', 'Placing Pea Gravel Filter', 'Placing CC Block', 'Ancilary Work for Site Completion'] volumes=['Percent', 'Nos', 'Nos', 'Percent', 'Cum', 'Cum', 'sqm', 'Cum', 'Nos', 'Percent'] codes=[100, 21058, 55449, 100, 7388, 918, 10767, 917, 76507, 100]
unit resources=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 210.0] total_consumption=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.5, 4]
unit resources=[0.0013333333333333333, 0.0, 0.0, 1.0, 0.1384122471532053, 0.01201495200982685, 0.02635537860220083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04, 12.0] total_consumption=[1.3373333333333333, 0.0, 0.0, 1003.0, 13

(10, 7)
(1, 2)
(1, 3)
(2, 4)
(3, 4)
(1, 5)
(4, 6)
(5, 6)
(4, 7)
(5, 7)
(4, 8)
(5, 8)
(4, 9)
(5, 9)
(9, 10)
[1, 5, 3, 2, 4, 9, 10, 8, 7, 6, 0]
1 2 4 6
1 2 4 7
1 2 4 8
1 2 4 9 10
1 3 4 6
1 3 4 7
1 3 4 8
1 3 4 9 10
1 5 6
1 5 7
1 5 8
1 5 9 10
[[1, 2, 4, 6], [1, 2, 4, 7], [1, 2, 4, 8], [1, 2, 4, 9, 10], [1, 3, 4, 6], [1, 3, 4, 7], [1, 3, 4, 8], [1, 3, 4, 9, 10], [1, 5, 6], [1, 5, 7], [1, 5, 8], [1, 5, 9, 10]]
Task-1 sdate=1
Task-2 sdate=8
Task-3 sdate=8
Task-4 sdate=29
Task-5 sdate=8
Task-6 sdate=50
Task-7 sdate=50
Task-8 sdate=50
Task-9 sdate=50
Task-10 sdate=67
75
unit=['Mobilization and Site Preparation', 'CC Block Manufacture(30X30X30)', 'CC Block Manufacture(40X40X20)', 'Curing CC Block', 'Embankment Construction', 'Placing Sand Filter', 'Placing Geotextile Filter', 'Placing Pea Gravel Filter', 'Placing CC Block', 'Ancilary Work for Site Completion'] volumes=['Percent', 'Nos', 'Nos', 'Percent', 'Cum', 'Cum', 'sqm', 'Cum', 'Nos', 'Percent'] codes=[100, 21058, 55449, 100, 7388, 918, 1076

...

(6, 7)
(1, 2)
(2, 3)
(2, 4)
(2, 5)
(5, 6)
[1, 2, 5, 6, 4, 3, 0]
1 2 3
1 2 4
1 2 5 6
[[1, 2, 3], [1, 2, 4], [1, 2, 5, 6]]
Task-1 sdate=1
Task-2 sdate=8
Task-3 sdate=23
Task-4 sdate=23
Task-5 sdate=23
Task-6 sdate=38
46
unit=['   Mobilization and Site Preparation', '   Embankment Construction', '   Placing Geotextile Filter', 'Geo Bag Placing(125 Kg)', 'Close Turfing', 'Ancilary Work for Site Completion'] volumes=['percent', 'Cum', 'sqm', 'Nos', 'percent', 'Percent'] codes=[100, 2479, 3400, 840, 2896, 100]
unit resources=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 210.0] total_consumption=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.5, 4]
unit resources=[0.0, 0.0033, 0.0013157894736842105, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 60.0] total_consumption=[0.0, 0.5478, 0.21842105263157893, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.6, 10]
unit resources=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.

(6, 7)
(1, 2)
(2, 3)
(2, 4)
(2, 5)
(5, 6)
[1, 2, 5, 6, 4, 3, 0]
1 2 3
1 2 4
1 2 5 6
[[1, 2, 3], [1, 2, 4], [1, 2, 5, 6]]
Task-1 sdate=1
Task-2 sdate=8
Task-3 sdate=23
Task-4 sdate=23
Task-5 sdate=23
Task-6 sdate=38
46
unit=['   Mobilization and Site Preparation', '   Embankment Construction', '   Placing Geotextile Filter', 'Geo Bag Placing(125 Kg)', 'Close Turfing', 'Ancilary Work for Site Completion'] volumes=['percent', 'Cum', 'sqm', 'Nos', 'percent', 'Percent'] codes=[100, 4166, 5716, 1412, 4868, 100]
unit resources=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 210.0] total_consumption=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.5, 4]
unit resources=[0.0, 0.0033, 0.0013157894736842105, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 60.0] total_consumption=[0.0, 0.9174, 0.3657894736842105, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 27.8, 17]
unit resources=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.

(6, 7)
(1, 2)
(2, 3)
(2, 4)
(2, 5)
(5, 6)
[1, 2, 5, 6, 4, 3, 0]
1 2 3
1 2 4
1 2 5 6
[[1, 2, 3], [1, 2, 4], [1, 2, 5, 6]]
Task-1 sdate=1
Task-2 sdate=8
Task-3 sdate=23
Task-4 sdate=23
Task-5 sdate=23
Task-6 sdate=38
46
unit=['   Mobilization and Site Preparation', '   Embankment Construction', '   Placing Geotextile Filter', 'Geo Bag Placing(125 Kg)', 'Close Turfing', 'Ancilary Work for Site Completion'] volumes=['percent', 'Cum', 'sqm', 'Nos', 'percent', 'Percent'] codes=[100, 2502, 3433, 848, 2924, 100]
unit resources=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 210.0] total_consumption=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.5, 4]
unit resources=[0.0, 0.0033, 0.0013157894736842105, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 60.0] total_consumption=[0.0, 0.5511, 0.21973684210526315, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 16.7, 11]
unit resources=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.

(6, 7)
(1, 2)
(2, 3)
(2, 4)
(2, 5)
(5, 6)
[1, 2, 5, 6, 4, 3, 0]
1 2 3
1 2 4
1 2 5 6
[[1, 2, 3], [1, 2, 4], [1, 2, 5, 6]]
Task-1 sdate=1
Task-2 sdate=8
Task-3 sdate=23
Task-4 sdate=23
Task-5 sdate=23
Task-6 sdate=26
34
unit=['   Mobilization and Site Preparation', '   Embankment Construction', '   Placing Geotextile Filter', 'Geo Bag Placing(125 Kg)', 'Close Turfing', 'Ancilary Work for Site Completion'] volumes=['percent', 'Cum', 'sqm', 'Nos', 'percent', 'Percent'] codes=[100, 355, 486, 120, 414, 100]
unit resources=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 210.0] total_consumption=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.5, 4]
unit resources=[0.0, 0.0033, 0.0013157894736842105, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 60.0] total_consumption=[0.0, 0.07919999999999999, 0.031578947368421054, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.4000000000000004, 2]
unit resources=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

(16, 7)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(2, 6)
(3, 6)
(4, 6)
(5, 6)
(1, 7)
(1, 8)
(7, 9)
(9, 10)
(9, 11)
(9, 12)
(9, 13)
(9, 14)
(14, 15)
(15, 16)
[1, 8, 7, 9, 14, 15, 16, 13, 12, 11, 10, 5, 4, 3, 2, 6, 0]
1 2 6
1 3 6
1 4 6
1 5 6
1 7 9 10
1 7 9 11
1 7 9 12
1 7 9 13
1 7 9 14 15 16
1 8
[[1, 2, 6], [1, 3, 6], [1, 4, 6], [1, 5, 6], [1, 7, 9, 10], [1, 7, 9, 11], [1, 7, 9, 12], [1, 7, 9, 13], [1, 7, 9, 14, 15, 16], [1, 8]]
Task-1 sdate=1
Task-2 sdate=8
Task-3 sdate=8
Task-4 sdate=8
Task-5 sdate=8
Task-6 sdate=19
Task-7 sdate=8
Task-8 sdate=8
Task-9 sdate=13
Task-10 sdate=28
Task-11 sdate=28
Task-12 sdate=28
Task-13 sdate=28
Task-14 sdate=28
Task-15 sdate=37
Task-16 sdate=42
50
unit=['   Mobilization and Site Preparation', '      CC Block Manufacture(50X50X50)', '      CC Block Manufacture(50X50X30)', '      CC Block Manufacture(40X40X40)', '      CC Block Manufacture(40X40X20)', 'Curing', '   Ring Bundh construction ', '   Excavation', '   Embankment Construction', '   Placing Sand Filter', '   

In [12]:
max_val=findMaximumDurations(resoureces_frames)
combined_resource_df=buildCombinedResourceMatrix(resoureces_frames)
print("max val={}".format(max_val))
combined_resource_df
resoureces_frames.append(combined_resource_df)
resources_sheet_names.append("Combined")
sheetName="Project_name"
name_df=pd.read_excel(inputpath,sheet_name=sheetName)
#project_name="Embankment Slope Protection Work at Dharmapasha Ruibbe Sunam/PW-07"
project_name=str(name_df.iloc[0,0])
filename=os.path.join(myfolder,"Total Resources Allocation.pdf")
#writeTaskAllocationReportCombined(project_name,task_df,filename,max_val)
writeResourceAllocationReportSummary(project_name,combined_resource_df,filename,max_val)

shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 34)
shape=(15, 65)
shape=(15, 65)
shape=(15, 65)
shape=(15, 65)
shape=(15, 65)
max=96
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 96)
shape=(15, 34)
shape=(15, 65)
shape=(15, 65)
shape=(15, 65)
shape=(15, 65)
shape=(15, 65)
max=96
days=['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31', 'D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38', 'D39', 'D40', 'D41', 'D42', 'D43', 'D44', 'D45', 'D46', 'D47', 'D48', 'D49', 'D50', 'D51', 'D52', 'D53', 'D54', 'D55', 'D56', 'D57', 'D58', 'D59', 'D60', 'D61', 'D62', 'D63', 'D64', 'D65', 'D66', 'D67', 'D68', 'D6

days=['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31', 'D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38', 'D39', 'D40', 'D41', 'D42', 'D43', 'D44', 'D45', 'D46', 'D47', 'D48', 'D49', 'D50', 'D51', 'D52', 'D53', 'D54', 'D55', 'D56', 'D57', 'D58', 'D59', 'D60', 'D61', 'D62']
days=['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31', 'D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38', 'D39', 'D40', 'D41', 'D42', 'D43', 'D44', 'D45', 'D46', 'D47', 'D48', 'D49', 'D50', 'D51', 'D52', 'D53', 'D54', 'D55', 'D56', 'D57', 'D58', 'D59', 'D60', 'D61', 'D62']
days=['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', '

wrting daily resources allocation data
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 594.0, 6974.0, 594.0, 49511.0, 0.0, 0.0, 627.0, 198.0]
['', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 594.0, 6974.0, 594.0, 49511.0, 0.0, 0.0, 627.0, 198.0]
wrting daily resources allocation data
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 594.0, 6974.0, 594.0, 49511.0, 0.0, 0.0, 627.0, 198.0]
['', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 594.0, 6974.0, 594.0, 49511.0, 0.0, 0.0, 627.0, 198.0]
wrting daily resources allocation data
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 594.0, 6974.0, 594.0, 49511.0, 0.0, 0.0, 627.0, 198.0]
['', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 594.0, 6974.0, 594.0, 49511.0, 0.0, 0.0, 627.0, 198.0]
wrting daily resources allocation data
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 594.0, 6974.0, 594.0, 49511.0, 0.0, 0.0, 627.0, 198.0]
['', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 594.0, 6974.0, 594.0, 49511.0, 0.0, 0.0, 627.0, 198.0]
wrting daily resources allocation data
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 594.0, 6974.0, 594.0, 495

In [13]:
saveDataFrame(myframes,outputpath_for_dframe,mynames)
saveDataFrame(resoureces_frames,outpath_for_resources,resources_sheet_names)  

In [14]:
unit_resource_df


,Code,Name,Unit,NS101,40-190-05,40-190-15,40-190-35,40-190-40,40-190-50,NS105,...,40-500-40,40-530-30,NS106,NS102,16-510,40-450,16-470,40-380-20,48-100,40-670-30
0,101,Mixture Machine,Nos,0.0,0.003704,0.006173,0.003155,0.001580,0.001333,0.0,...,0.000,0.00,0.000000,0.0,0.0000,0.0000,0.0000,0.000000,0.00,0.000
1,102,Excavator,Nos,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000,0.00,0.000000,0.0,0.0033,0.0033,0.0033,0.003300,0.00,0.000
2,103,Compaction Roller,Nos,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000,0.00,0.000000,0.0,0.0000,0.0000,0.0000,0.000000,0.00,0.000
3,104,Form Work,Nos,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,...,0.000,0.00,0.000000,0.0,0.0000,0.0000,0.0000,0.000000,0.00,0.000
4,201,Cement,Bag,0.0,0.384478,0.640797,0.328088,0.164044,0.138412,0.0,...,0.000,0.00,0.000000,0.0,0.0000,0.0000,0.0000,0.000000,0.00,0.210
5,202,Sand 1.5 FM,cum,0.0,0.033375,0.055625,0.028480,0.014240,0.012015,0.0,...,0.000,0.00,0.000000,0.0,0.0000,0.0000,0.0000,0.000000,0.00,0.031
6,203,Stone Chips,cum,0.0,0.073209,0.122016,0.062472,0.031236,0.026355,0.0,...,0.000,0.00,0.000000,0.0,0.0000,0.0000,0.0000,0.000000,0.00,0.000
7,204,Filter Sand,cum,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000,0.00,0.000000,0.0,0.0000,0.0000,0.0000,0.000000,0.00,0.000
8,205,Geotextile Filter,sqm,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,1.000,0.00,0.000000,0.0,0.0000,0.0000,0.0000,0.000000,0.00,0.000
9,206,Peagravel,cum,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000,1.00,0.000000,0.0,0.0000,0.0000,0.0000,0.000000,0.00,0.000
